<a href="https://colab.research.google.com/github/anajuliab/pipeline_ETL_santander_bootcamp/blob/main/Projeto_DIO_ETL_Bootcamp_Santander_adaptado_DevWeek2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Santander Dev Week 2023 (ETL com Python): Bootcamp Santander - Projeto adaptado

**Contexto:** O desafio para esse projeto seria pensar em um novo pipeline ETL.
Dessa forma, pensei em gerar uma chave pix aleatória para o meu usuário.

**Condições do Problema:**

1. Você recebeu uma planilha simples, em formato CSV ('NOME_PLANILHA.csv'), com uma lista de IDs de usuário do banco:
  ```
  UserID
  1
  2
  3
  4
  5
  ```
2. Seu trabalho é consumir o endpoint `GET https://sdw-2023-prd.up.railway.app/users/{id}` (API da Santander Dev Week 2023) para obter os dados de cada cliente.
3. Depois de obter os dados dos clientes, você vai gerar uma chave pix aleatória, de 32 caracteres (dentre eles letras e números).
4. Uma vez que a chave pix para cliente esteja pronta, você vai enviar essas informações de volta para a API, atualizando a lista de "news" de cada usuário usando o endpoint `PUT https://sdw-2023-prd.up.railway.app/users/{id}`.



In [28]:
# Repositório do Projeto: https://github.com/anajuliab/pipeline_ETL_santander_bootcamp
# Repositório da API: https://github.com/digitalinnovationone/santander-dev-week-2023-api
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

## **E**xtract

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.

In [29]:
import pandas as pd

df = pd.read_csv('desafio.csv')
user_ids = df['User ID'].tolist()
print(user_ids)

[1718, 1719, 1720]


In [30]:
import requests
import json

def get_user(id):
  response = requests.get(f'{sdw2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

[
  {
    "id": 1718,
    "name": "Pytherson",
    "account": {
      "id": 1804,
      "number": "51111-1",
      "agency": "0001",
      "balance": 0.0,
      "limit": 2000.0
    },
    "card": {
      "id": 1659,
      "number": "**** **** 4444 3333",
      "limit": 1000.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 1719,
    "name": "Pip",
    "account": {
      "id": 1805,
      "number": "51112-1",
      "agency": "0001",
      "balance": 0.0,
      "limit": 2000.0
    },
    "card": {
      "id": 1660,
      "number": "**** **** 4445 3333",
      "limit": 1000.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 1720,
    "name": "Pep",
    "account": {
      "id": 1806,
      "number": "51113-1",
      "agency": "0001",
      "balance": 0.0,
      "limit": 2000.0
    },
    "card": {
      "id": 1661,
      "number": "**** **** 4446 3333",
      "limit": 1000.0
    },
    "features": [],
    "news": []
  }
]


## **T**ransform

Diferente do que foi exempificado na aula da DIO, eu vou criar uma sequência de dígitos e letras aleatórios (random) para criar uma chave pix aleatória.
Como sabemos que a chave pix tem 32 caracteres, dentre eles números e letras, podemos utilizar essa informação para gerá-la.

Penso em futuramente gerar um cartão online, com número verificador e data de vencimento.

In [31]:
import string
import random

def random_chave_pix(size=32, chars=string.ascii_uppercase + string.digits):
 return ''.join(random.choice(chars) for _ in range(size))



for user in users:
  news = random_chave_pix()
  print(news)
  user['news'].append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
      "description": news
  })


JWAEEEFPHFHI6H4IVXORKK6XRUY7OLSB
A7GN2PQXXGI8L5SZBRVVO1TFGCQY215Y
XVNWAQ5DI1CUE8UNF93DXS9GYAEO2QWU


## **L**oad

Atualize a lista de "news" de cada usuário na API com a nova chave pix gerada.

In [32]:
def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")

User Pytherson updated? True!
User Pip updated? True!
User Pep updated? True!
